In [ ]:
import pandas as pd

# Carrega a base
df = pd.read_excel("Case - Wpp_PIPELINE_CLASSIFICADO_final(1).xlsx")

# 1. Padroniza o texto das colunas-chave para garantir a comparação
for col in ['Classificacao_Objetivo', 'Classificacao_Aporte', 'Classificacao_Resgate']:
    # Converte para string e para MAIÚSCULA, transformando nulos em 'NAN'
    df[col] = df[col].astype(str).str.upper() 

# LÓGICA DE FILTRAGEM 
# Vamos criar uma máscara booleana para identificar linhas 'inválidas'
# Uma linha é inválida se *todos* os campos forem 'NÃO SE APLICA' ou 'NAN'

# Condição A: A célula é 'NÃO SE APLICA'
condicao_nao_se_aplica = (
    (df['Classificacao_Objetivo'] == "NÃO SE APLICA") &
    (df['Classificacao_Aporte'] == "NÃO SE APLICA") &
    (df['Classificacao_Resgate'] == "NÃO SE APLICA")
)

# Condição B: A célula é 'NAN' (vazio)
condicao_nan = (
    (df['Classificacao_Objetivo'] == "NAN") &
    (df['Classificacao_Aporte'] == "NAN") &
    (df['Classificacao_Resgate'] == "NAN")
)

# Mantém apenas as linhas que NÃO atendem à Condição A E à Condição B
df_filtrado = df[~condicao_nao_se_aplica & ~condicao_nan].copy()
# ==========================================

print(f"Tamanho do DF filtrado após flexibilização: {len(df_filtrado)} linhas.")

# ... O restante do código de agrupamento e mapeamento de perfis continua o mesmo ...

# Agrupa por objetivo, aporte e resgate
agrupado = (
    df_filtrado
    .groupby(['Classificacao_Objetivo', 'Classificacao_Aporte', 'Classificacao_Resgate'])
    .size()
    .reset_index(name='Contagem')
)

# Ordena para ver combinações mais comuns
agrupado = agrupado.sort_values('Contagem', ascending=False)

# === Mapeamento de perfis (Continua o mesmo) ===
def definir_perfil(objetivo, aporte, resgate):
    objetivo = str(objetivo).upper()
    aporte = str(aporte).upper()
    resgate = str(resgate).upper()
    
    # APRENDIZADO
    if "APRENDIZADO" in objetivo:
        if "BAIXO" in aporte:
            return "Investidor Conservador CP"
        else:
            return "Investidor Conservador CP"

    # RENDA PASSIVA
    elif "RENDA PASSIVA" in objetivo:
        if "ALTO" in aporte and "NÃO PRETENDE" in resgate:
            return "Investidor Moderado LP"
        elif "MÉDIO" in aporte and "PRETENDE" in resgate:
            return "Investidor Moderado LP"
        else:
            return "Investidor Moderado LP"

    # APOSENTADORIA
    elif "APOSENTADORIA" in objetivo:
        if "ALTO" in aporte and "NÃO PRETENDE" in resgate:
            return "Investidor Conservador LP"
        elif "MÉDIO" in aporte:
            return "Investidor Conservador LP"
        else:
            return "Investidor Conservador LP"
        
    #RELACIONAMENTO
    elif "RELACIONAMENTO" in objetivo:
        if "ALTO" in aporte and "NÃO PRETENDE" in resgate:
            return "Investidor Conservador CP"
        elif "MÉDIO" in aporte:
            return "Investidor Conservador CP"
        else:
            return "Investidor Conservador CP"
        
    # RESERVA / SEGURANÇA
    elif "RESERVA" in objetivo or "SEGURANÇA" in objetivo:
        if "BAIXO" in aporte and "PRETENDE" in resgate:
            return "Investidor Conservador CP"
        else:
            return "Investidor Conservador CP"

    # CRESCIMENTO / ACÚMULO
    elif "ACÚMULO" in objetivo or "CRESCIMENTO" in objetivo:
        if "ALTO" in aporte and "NÃO PRETENDE" in resgate:
            return "Investidor Moderado LP"
        else:
            return "Investidor Moderado LP"

    # TRADE
    elif "TRADE" in objetivo or "MERCADO VARIÁVEL" in objetivo:
        if "ALTO" in aporte:
            return "Investidor Arrojado"
        else:
            return "Investidor Arrojado"

    # OBJETIVOS ESPECÍFICOS
    elif "OBJETIVOS ESPECÍFICOS" in objetivo:
        if "MÉDIO" in aporte:
            return "Investidor Moderado MP"
        else:
            return "Investidor Moderado MP"

    else:
        return "Não Classificado"

# Aplica classificação no resultado agregado
agrupado['Perfil_Cliente'] = agrupado.apply(
    lambda row: definir_perfil(
        row['Classificacao_Objetivo'],
        row['Classificacao_Aporte'],
        row['Classificacao_Resgate']
    ),
    axis=1
)

# Exporta base agregada
agrupado.to_excel("base_perfis_agrupados.xlsx", index=False)

print("✅ Base de perfis agregados gerada com sucesso: base_perfis_agrupados.xlsx")
